# ACE-Step 1.5 - One-Click Colab Server
Click the play button below to start the server. Once it finishes, look for the **gradio.live** link at the bottom and paste it into your local Thin Client.

In [ ]:
# @title Start ACE-Step 1.5 Server
import os

# @markdown --- 
# @markdown **Security Settings**
API_KEY = "" # @param {type:"string"}

print("1/4 Cloning repository...")
!git clone https://github.com/AgentHitmanFaris/ACE-Step-1.5-GCollab-
%cd ACE-Step-1.5-GCollab-

print("\n2/4 Installing dependencies...")
!pip install -r requirements.txt
!pip install diffusers accelerate transformers gradio modelscope soundfile einops vector-quantize-pytorch torchcodec loguru
!pip install -e acestep/third_parts/nano-vllm

print("\n3/4 Downloading models (this depends on your connection speed)...")
!python -m acestep.model_downloader

print("\n4/4 Launching ACE-Step server...")
cmd = "python -m acestep.acestep_v15_pipeline --enable-api --share"
if API_KEY:
  cmd += f" --api-key {API_KEY}"
!{cmd}